# Evaluation of all experiments

## Imports

In [4]:
import os 
import numpy as np
import pandas as pd
from itertools import product
from typing import List

from utils.evaluation_methods import (generate_all_combinations,
                                     evaluate_run, 
                                     prepare_params_table_export,
                                     evaluate_peft_run,
                                     create_empty_df,
                                     create_empty_df_params,
                                     get_best_results_comp_tun,
                                     evaluate_components_tuning_run)

## Settings

In [5]:
"""
    models: all models used for imputation task
    datasets: all datasets used for training and evaluating imputation models
    mask_ratios: ratios for randomly masking the time series
    
"""
exp_models = [
    'Gpt',
    'Bert',
    'Saits',
    'TimesNet',
    'Transformer',
    'Llama',
    'Phi',
    'T5',
    'Bart'
]

exp_datasets = [
    'ETTh1',
    'ETTh2',
    'ETTm1',
    'ETTm2',
    'Weather',
    'Electricity',
]

exp_mask_ratios = [
    '0.125',
    '0.25',
    '0.375',
    '0.5',
]

# Evaluation

In [6]:
list_experiments = generate_all_combinations(models=exp_models, datasets=exp_datasets, mask_ratios=exp_mask_ratios)

## RQ1: Comparison of all models

In [7]:
run_name_final = "rq1_llm_comparison"

In [8]:
final_run_complete, final_run_agg, final_run_params = evaluate_run(run_name=run_name_final, 
                                                                   experiments=list_experiments, 
                                                                   datasets=exp_datasets, 
                                                                   mask_ratios=exp_mask_ratios, 
                                                                   models=exp_models)

In [9]:
final_run_params[:1]

Gpt                                    Bert                   \
       all_params trainable_params percentage  all_params trainable_params   
ETTh1  60759552.0         906240.0       1.49  45838848.0         551424.0   

                         Llama                                      Phi  \
      percentage    all_params trainable_params percentage   all_params   
ETTh1        1.2  1752240128.0        2162688.0       0.12  761758720.0   

                                           T5                              \
      trainable_params percentage  all_params trainable_params percentage   
ETTh1        1351680.0       0.18  74676416.0         456128.0       0.61   

             Bart                              
       all_params trainable_params percentage  
ETTh1  73556736.0        1883136.0       2.56

In [10]:
elements_to_remove = {"Saits", "TimesNet", "Transformer"}
final_run_params_exp = prepare_params_table_export(index=[model for model in exp_models if model not in elements_to_remove], parameters=final_run_params)
final_run_params_exp

,all_params,trainable_params,percentage
Gpt,60759552.0,906240.0,1.49
Bert,45838848.0,551424.0,1.2
Llama,1752240128.0,2162688.0,0.12
Phi,761758720.0,1351680.0,0.18
T5,74676416.0,456128.0,0.61
Bart,73556736.0,1883136.0,2.56


In [11]:
final_run_complete

Gpt                Bert               Saits  \
                             MSE       MAE       MSE       MAE       MSE   
Dataset     Mask Ratio                                                     
ETTh1       0.125       0.052361  0.154249  0.046628  0.146125  0.027698   
            0.25        0.073752  0.182132  0.063819  0.169613  0.039232   
            0.375       0.097901  0.207094  0.083703  0.194175  0.046693   
            0.5         0.122755  0.231671  0.106765   0.21625  0.063742   
ETTh2       0.125       0.041419  0.130988  0.039527  0.127584  0.062706   
            0.25        0.049646  0.146476  0.045757  0.140556  0.061057   
            0.375       0.056733  0.157836  0.051942  0.151648  0.091486   
            0.5         0.066571   0.17175  0.060543  0.164902  0.107946   
ETTm1       0.125       0.024353  0.104534  0.019583   0.09305  0.015104   
            0.25        0.032868  0.121154  0.023799  0.102897  0.016345   
            0.375       0.041288   0.13556  0.029749  0.115068  0.018385   
            0.5         0.053334  0.152863  0.036387   0.12774  0.028606   
ETTm2       0.125       0.020909  0.087161   0.01851  0.080245  0.022794   
            0.25        0.025017  0.097023  0.020551  0.085497  0.030417   
            0.375       0.028179  0.104417  0.023324  0.093001   0.04581   
            0.5         0.031337  0.111245   0.02642  0.099741  0.055998   
Electricity 0.125       0.071642  0.186019  0.071316  0.185704  0.145683   
            0.25        0.079975  0.196334  0.079491  0.195558  0.160686   
            0.375        0.08862  0.206764  0.087274  0.204883  0.171591   
            0.5         0.096115  0.216195  0.096293  0.215828  0.181068   
Weather     0.125       0.025955  0.047594  0.026854  0.049629  0.028483   
            0.25        0.029656  0.054529   0.02953  0.054724   0.03059   
            0.375       0.032986  0.060237  0.032448  0.059123  0.049832   
            0.5         0.037267  0.066907  0.037405  0.066181   0.03851   

                                  TimesNet           Transformer            \
                             MAE       MSE       MAE         MSE       MAE   
Dataset     Mask Ratio                                                       
ETTh1       0.125       0.111945  0.067687  0.176257    0.058845  0.172224   
            0.25        0.131963  0.084629  0.195001      0.0821  0.203554   
            0.375       0.142504  0.099912   0.21213    0.107571  0.231826   
            0.5         0.167715  0.120807  0.231209    0.134972  0.261142   
ETTh2       0.125       0.171265  0.041293  0.136248    0.168201  0.298816   
            0.25        0.172384  0.048219  0.147093    0.228018  0.347638   
            0.375       0.210616  0.054247  0.156121    0.287346  0.391217   
            0.5         0.231629  0.062368  0.166747      0.3106  0.409541   
ETTm1       0.125       0.082191  0.019601  0.093129    0.021112  0.101057   
            0.25        0.083928  0.023927  0.102251    0.027695  0.116453   
            0.375       0.087755  0.029652  0.114066    0.034537  0.129623   
            0.5         0.113986  0.036918  0.126275    0.043517   0.14621   
ETTm2       0.125       0.096701  0.019473  0.083312    0.150361  0.284269   
            0.25        0.116755  0.021137  0.087198    0.156736  0.282212   
            0.375       0.147005  0.023008  0.093068    0.225575  0.345279   
            0.5         0.165699  0.025784  0.098864    0.221929  0.346564   
Electricity 0.125        0.27184  0.089107  0.204203    0.146989  0.275979   
            0.25        0.281293  0.096658  0.211289    0.158752  0.282719   
            0.375       0.287883  0.098319  0.215152    0.165015  0.287213   
            0.5         0.294727  0.103628  0.222129    0.179126  0.297319   
Weather     0.125       0.064073  0.026032  0.047556    0.032072  0.086743   
            0.25        0.069294  0.028588  0.052874    0.034371  0.083639   
            0.375       0.

In [12]:
final_run_agg

Gpt                Bert               Saits            \
                  MSE       MAE       MSE       MAE       MSE       MAE   
Dataset                                                                   
ETTh1        0.086692  0.193787  0.075229  0.181541  0.044341  0.138532   
ETTh2        0.053592  0.151762  0.049442  0.146173  0.080799  0.196474   
ETTm1        0.037961  0.128528  0.027379  0.109689   0.01961  0.091965   
ETTm2         0.02636  0.099961  0.022201  0.089621  0.038755   0.13154   
Electricity  0.084088  0.201328  0.083593  0.200493  0.164757  0.283936   
Weather      0.031466  0.057317  0.031559  0.057414  0.036854   0.08309   

             TimesNet           Transformer               Llama            \
                  MSE       MAE         MSE       MAE       MSE       MAE   
Dataset                                                                     
ETTh1        0.093259  0.203649    0.095872  0.217187  0.063538  0.167257   
ETTh2        0.051532  0.151552    0.248541  0.361803  0.050797  0.147028   
ETTm1        0.027525   0.10893    0.031715  0.123336  0.026839  0.104978   
ETTm2        0.022351   0.09061     0.18865  0.314581   0.02466  0.094746   
Electricity  0.096928  0.213193     0.16247  0.285808  0.080362  0.196474   
Weather      0.030466  0.055481    0.036066  0.086959  0.033024   0.05977   

                  Phi                  T5                Bart            
                  MSE       MAE       MSE       MAE       MSE       MAE  
Dataset                                                                  
ETTh1        0.065666  0.171606  0.077256  0.186871  0.081629  0.190011  
ETTh2        0.049977    0.1448   0.04903  0.146522  0.050359  0.148358  
ETTm1        0.027038  0.105929  0.033057   0.12153  0.030914  0.116936  
ETTm2        0.025685   0.09686  0.024046  0.095434  0.024109  0.095101  
Electricity  0.079935  0.195572   0.09023  0.209161  0.087876   0.20512  
Weather      0.083383  0.129502  0.030412  0.056445  0.035474  0.067048

## RQ1: Comparison of LLM Methods

In [30]:
final_run_complete_llms_df = final_run_complete[[("Gpt", "MSE"),
                                             ("Gpt", "MAE"),
                                             ("Bert", "MSE"),
                                             ("Bert", "MAE"),
                                             ("Llama", "MSE"),
                                             ("Llama", "MAE"),
                                             ("Phi", "MSE"),
                                             ("Phi", "MAE"),
                                             ("T5", "MSE"),
                                             ("T5", "MAE"),
                                             ("Bart", "MSE"),
                                             ("Bart", "MAE")]]
final_run_complete_llms_df

Gpt                Bert               Llama  \
                             MSE       MAE       MSE       MAE       MSE   
Dataset     Mask Ratio                                                     
ETTh1       0.125       0.052361  0.154249  0.046628  0.146125  0.040863   
            0.25        0.073752  0.182132  0.063819  0.169613  0.051621   
            0.375       0.097901  0.207094  0.083703  0.194175  0.068943   
            0.5         0.122755  0.231671  0.106765   0.21625  0.092727   
ETTh2       0.125       0.041419  0.130988  0.039527  0.127584   0.04073   
            0.25        0.049646  0.146476  0.045757  0.140556  0.047797   
            0.375       0.056733  0.157836  0.051942  0.151648  0.053168   
            0.5         0.066571   0.17175  0.060543  0.164902  0.061494   
ETTm1       0.125       0.024353  0.104534  0.019583   0.09305  0.019267   
            0.25        0.032868  0.121154  0.023799  0.102897  0.023253   
            0.375       0.041288   0.13556  0.029749  0.115068  0.028768   
            0.5         0.053334  0.152863  0.036387   0.12774  0.036067   
ETTm2       0.125       0.020909  0.087161   0.01851  0.080245  0.020956   
            0.25        0.025017  0.097023  0.020551  0.085497  0.023572   
            0.375       0.028179  0.104417  0.023324  0.093001  0.024871   
            0.5         0.031337  0.111245   0.02642  0.099741  0.029243   
Electricity 0.125       0.071642  0.186019  0.071316  0.185704  0.064977   
            0.25        0.079975  0.196334  0.079491  0.195558  0.077003   
            0.375        0.08862  0.206764  0.087274  0.204883  0.085248   
            0.5         0.096115  0.216195  0.096293  0.215828  0.094221   
Weather     0.125       0.025955  0.047594  0.026854  0.049629  0.026511   
            0.25        0.029656  0.054529   0.02953  0.054724  0.031033   
            0.375       0.032986  0.060237  0.032448  0.059123  0.035205   
            0.5         0.037267  0.066907  0.037405  0.066181  0.039349   

                                       Phi                  T5            \
                             MAE       MSE       MAE       MSE       MAE   
Dataset     Mask Ratio                                                     
ETTh1       0.125       0.137663   0.04152   0.13933  0.051325  0.155268   
            0.25        0.152756  0.055132  0.159479  0.068824  0.177943   
            0.375       0.174991  0.070479   0.17962  0.083984  0.195856   
            0.5         0.203617  0.095533  0.207997   0.10489  0.218416   
ETTh2       0.125       0.130699  0.040965  0.129862  0.039729  0.130664   
            0.25        0.142798  0.045841  0.138333  0.046804  0.143603   
            0.375       0.151732  0.052486  0.149489  0.051694  0.151058   
            0.5         0.162882  0.060617  0.161516  0.057893  0.160762   
ETTm1       0.125       0.091064  0.018845  0.090396  0.024139  0.104921   
            0.25        0.098516  0.022587  0.097936  0.030412  0.116839   
            0.375       0.108964  0.028593  0.109421  0.035589  0.126424   
            0.5         0.121366  0.038127  0.125962  0.042085  0.137936   
ETTm2       0.125       0.087224  0.022621  0.089739  0.020454  0.086771   
            0.25        0.091686  0.023094    0.0914   0.02257  0.091715   
            0.375       0.095256  0.026891  0.099593  0.024965  0.098049   
            0.5         0.104819  0.030134  0.106707  0.028197  0.105199   
Electricity 0.125       0.176613  0.065353  0.176924   0.08039  0.197653   
            0.25        0.192349  0.075477  0.190072  0.086734  0.204747   
            0.375       0.203006  0.084636  0.201788  0.093323  0.212719   
            0.5          0.21393  0.094272  0.213503  0.100473  0.221527   
Weather     0.125       0.048029  0.080304  0.124188  0.026134  0.049699   
            0.25        0.056592  0.082929  0.129816  0.029081  0.054045   
            0.375        0.06447  0.086615   0.13686   0.03156  0.058816   


In [14]:
final_run_sum_llms_df = final_run_agg[[("Gpt", "MSE"),
                                     ("Gpt", "MAE"),
                                     ("Bert", "MSE"),
                                     ("Bert", "MAE"),
                                     ("Llama", "MSE"),
                                     ("Llama", "MAE"),
                                     ("Phi", "MSE"),
                                     ("Phi", "MAE"),
                                     ("T5", "MSE"),
                                     ("T5", "MAE"),
                                     ("Bart", "MSE"),
                                     ("Bart", "MAE")]]
final_run_sum_llms_df

Gpt                Bert               Llama            \
                  MSE       MAE       MSE       MAE       MSE       MAE   
Dataset                                                                   
ETTh1        0.086692  0.193787  0.075229  0.181541  0.063538  0.167257   
ETTh2        0.053592  0.151762  0.049442  0.146173  0.050797  0.147028   
ETTm1        0.037961  0.128528  0.027379  0.109689  0.026839  0.104978   
ETTm2         0.02636  0.099961  0.022201  0.089621   0.02466  0.094746   
Electricity  0.084088  0.201328  0.083593  0.200493  0.080362  0.196474   
Weather      0.031466  0.057317  0.031559  0.057414  0.033024   0.05977   

                  Phi                  T5                Bart            
                  MSE       MAE       MSE       MAE       MSE       MAE  
Dataset                                                                  
ETTh1        0.065666  0.171606  0.077256  0.186871  0.081629  0.190011  
ETTh2        0.049977    0.1448   0.04903  0.146522  0.050359  0.148358  
ETTm1        0.027038  0.105929  0.033057   0.12153  0.030914  0.116936  
ETTm2        0.025685   0.09686  0.024046  0.095434  0.024109  0.095101  
Electricity  0.079935  0.195572   0.09023  0.209161  0.087876   0.20512  
Weather      0.083383  0.129502  0.030412  0.056445  0.035474  0.067048

## RQ2: Components Tuning
Legende:
- Att = Attention
- Add & LN = Add & LayerNorm
- FFN = Feedforward Network 




In [15]:
exp_com_tun = [
    'attention',
    'add_layernorm',
    'ffn',
    'attention_ffn',
    'ffn_add_layernorm',
    'attention_add_layernorm',
    'attention_add_layernorm_ffn'    
]

helper_array = ["0125" , "025", "0375", "05"]
run_name_comp_tun = "rq2_components/components_tuning_3_layers_"

In [16]:
comp_tun_combinations = generate_all_combinations(datasets=exp_datasets, mask_ratios=helper_array, models=exp_com_tun)

In [17]:
df_comp_tun_comb = create_empty_df(datasets=exp_datasets, mask_ratios=exp_mask_ratios, models=exp_com_tun)
df_comp_tun_params = create_empty_df_params(datasets=exp_datasets, models=exp_com_tun)

In [18]:
comp_tun_results = []
for elem in helper_array:
    run_name_comp_tun_helper = f'{run_name_comp_tun}{elem}'
    results_directory = f'./results/{run_name_comp_tun_helper}'
    if os.path.exists(results_directory):
        folders = os.listdir(results_directory)
        experiments_com_tun = generate_all_combinations(datasets=exp_datasets, mask_ratios=[elem], models=exp_com_tun)
        tmp_results = get_best_results_comp_tun(experiments=experiments_com_tun, 
                                                folders=folders, 
                                                run_name=run_name_comp_tun_helper)
        comp_tun_results += tmp_results


comp_tun_results_df, comp_tun_results_df_agg, comp_tun_params_df = evaluate_components_tuning_run(empty_df=df_comp_tun_comb, empty_df_params=df_comp_tun_params, results=comp_tun_results)

In [19]:
comp_tun_results_df

attention           add_layernorm                 ffn  \
                             MSE       MAE           MSE       MAE       MSE   
Dataset     Mask Ratio                                                         
ETTh1       0.125       0.046464  0.145935      0.061842  0.163472  0.045603   
            0.25        0.066138  0.172822      0.086867  0.195558  0.059329   
            0.375       0.083015   0.19458      0.111773  0.220486  0.077195   
            0.5         0.111154  0.223658      0.141538  0.245964  0.111287   
ETTh2       0.125       0.040957  0.130443      0.043992  0.136131   0.04174   
            0.25        0.046503  0.141139      0.051935  0.150316  0.046867   
            0.375       0.055067   0.15475      0.059325  0.162498  0.054224   
            0.5          0.06274  0.166162      0.070043   0.17673   0.06101   
ETTm1       0.125       0.020304  0.094135      0.028807  0.113283  0.019059   
            0.25        0.026712  0.108424      0.039659  0.132195   0.02355   
            0.375        0.03402  0.122101      0.049133  0.147068  0.031151   
            0.5         0.045357  0.140464      0.063923  0.165963  0.043503   
ETTm2       0.125        0.02004  0.083929      0.022037  0.090134  0.019305   
            0.25        0.023252  0.092182      0.025908   0.09977  0.021558   
            0.375       0.025784   0.09824      0.029728  0.108341  0.023761   
            0.5         0.029953   0.10753      0.033828  0.116103  0.026606   
Electricity 0.125       0.072882  0.186878      0.072466  0.187292  0.077808   
            0.25        0.080524  0.196263      0.081492  0.198393  0.085978   
            0.375       0.087746   0.20496      0.089541  0.208527  0.094115   
            0.5         0.096313  0.215353      0.098157  0.219068  0.100347   
Weather     0.125       0.026035  0.046689      0.026264  0.047614  0.025465   
            0.25        0.028833  0.052577      0.031178  0.057695  0.029148   
            0.375       0.032082  0.058194      0.033247   0.06047  0.031698   
            0.5         0.036373  0.064522      0.037784  0.066791  0.035738   

                                 attention_ffn           ffn_add_layernorm  \
                             MAE           MSE       MAE               MSE   
Dataset     Mask Ratio                                                       
ETTh1       0.125       0.146453      0.041606  0.138763          0.041925   
            0.25         0.16431      0.053984  0.156031          0.054014   
            0.375       0.186728      0.072233  0.178588          0.074813   
            0.5         0.220705      0.107414  0.216352          0.108384   
ETTh2       0.125       0.130304        0.0394  0.126342           0.04052   
            0.25        0.138993      0.044331  0.134082          0.045955   
            0.375       0.151024      0.052208   0.14682          0.052638   
            0.5         0.160745      0.059688  0.158777          0.060465   
ETTm1       0.125        0.09157      0.017205  0.086215          0.018358   
            0.25        0.100283      0.022264  0.097281           0.02331   
            0.375       0.113494      0.029468  0.110612          0.030536   
            0.5         0.132845      0.040675  0.129155          0.042471   
ETTm2       0.125       0.079636      0.018162  0.077317          0.018659   
            0.25        0.085227      0.019826  0.081475          0.020949   
            0.375       0.090877      0.022274    0.0877          0.023455   
            0.5          0.09804      0.025486  0.095155          0.026669   
Electricity 0.125       0.191439      0.081532  0.195653          0.079626   
            0.25        0.202127      0.087194  0.202962          0.087131   
            0.375       0.210326      0.093312  0.210548          0.093685   
            0.5         0.218649      0.100817  0.220094          0.099988   
Weather     0.125       0.045691      0.025267  0.045058          0.

In [20]:
comp_tun_results_df_agg

attention           add_layernorm                 ffn            \
                  MSE       MAE           MSE       MAE       MSE       MAE   
Dataset                                                                       
ETTh1        0.076693  0.184249      0.100505   0.20637  0.073354  0.179549   
ETTh2        0.051317  0.148124      0.056324  0.156419   0.05096  0.145266   
ETTm1        0.031598  0.116281      0.045381  0.139627  0.029315  0.109548   
ETTm2        0.024758   0.09547      0.027875  0.103587  0.022807  0.088445   
Electricity  0.084366  0.200864      0.085414   0.20332  0.089562  0.205635   
Weather      0.030831  0.055496      0.032118  0.058143  0.030512  0.054475   

            attention_ffn           ffn_add_layernorm            \
                      MSE       MAE               MSE       MAE   
Dataset                                                           
ETTh1            0.068809  0.172434          0.069784  0.174268   
ETTh2            0.048907  0.141505          0.049894  0.143472   
ETTm1            0.027403  0.105816          0.028669  0.108325   
ETTm2            0.021437  0.085412          0.022433  0.087228   
Electricity      0.090714  0.207314          0.090108  0.206417   
Weather          0.030416  0.053832          0.030246  0.054162   

            attention_add_layernorm           attention_add_layernorm_ffn  \
                                MSE       MAE                         MSE   
Dataset                                                                     
ETTh1                      0.075288   0.18147                    0.069116   
ETTh2                      0.050625  0.146605                    0.049001   
ETTm1                      0.031116  0.115624                    0.028089   
ETTm2                      0.024136   0.09398                    0.021445   
Electricity                0.084306  0.200587                    0.091062   
Weather                    0.030632  0.055351                    0.030294   

                       
                  MAE  
Dataset                
ETTh1        0.173025  
ETTh2        0.141789  
ETTm1        0.107162  
ETTm2        0.085365  
Electricity  0.207484  
Weather      0.054034

In [21]:
comp_tuning_params_df_exp = prepare_params_table_export(index=exp_com_tun, parameters=comp_tun_params_df)

In [22]:
comp_tuning_params_df_exp

,all_params,trainable_params,percentage
attention,60648960.0,46470912.0,76.62
add_layernorm,60648960.0,39394560.0,64.96
ffn,60648960.0,53551104.0,88.3
attention_ffn,60648960.0,60638208.0,99.98
ffn_add_layernorm,60648960.0,53561856.0,88.31
attention_add_layernorm,60648960.0,46481664.0,76.64
attention_add_layernorm_ffn,60648960.0,60648960.0,100.0


## RQ3: PEFT Methods

In [23]:
exp_peft_methods = [
    'lora',
    'adalora',
    'ia3',
    'qlora',
]

helper_array_peft = ["0125" , "025", "0375", "05"]

In [24]:
experiments_peft = generate_all_combinations(datasets=exp_datasets, mask_ratios=helper_array_peft, models=exp_peft_methods)

In [25]:
peft_tuning_results, peft_tuning_results_agg, peft_params = evaluate_peft_run(experiments=experiments_peft, 
                                                                              datasets=exp_datasets, 
                                                                              mask_ratios=helper_array_peft, 
                                                                              mask_ratios_init=exp_mask_ratios,
                                                                              peft_methods=exp_peft_methods)

In [26]:
peft_tuning_results

lora             adalora                 ia3  \
                             MSE       MAE       MSE       MAE       MSE   
Dataset     Mask Ratio                                                     
ETTh1       0.125       0.052361  0.154249  0.053067  0.153815  0.062518   
            0.25        0.073752  0.182132  0.079252  0.186329  0.085761   
            0.375       0.097901  0.207094  0.103974  0.212342  0.111802   
            0.5         0.122755  0.231671  0.129739  0.236775  0.142089   
ETTh2       0.125        0.04207  0.131539  0.042822   0.13316  0.044389   
            0.25        0.049646  0.146476   0.04977    0.1468   0.05253   
            0.375       0.056733  0.157836  0.057921  0.159699  0.059941   
            0.5         0.066571   0.17175   0.06634  0.171631  0.068786   
ETTm1       0.125       0.024353  0.104534  0.026747  0.109564  0.029064   
            0.25        0.032868  0.121154  0.035833  0.126274  0.038409   
            0.375       0.041288   0.13556  0.045172  0.141076  0.050172   
            0.5         0.053334  0.152863  0.057729  0.158526  0.063243   
ETTm2       0.125       0.020909  0.087161  0.022232  0.090191  0.022603   
            0.25        0.025017  0.097023  0.024944  0.097456  0.026058   
            0.375       0.028179  0.104417  0.028549  0.105129  0.029637   
            0.5         0.031337  0.111245  0.032229  0.112828   0.03386   
Electricity 0.125       0.071642  0.186019  0.071377  0.185822  0.072327   
            0.25        0.079975  0.196334  0.080721   0.19751  0.081725   
            0.375        0.08862  0.206764  0.088217  0.206543  0.089831   
            0.5         0.096115  0.216195  0.096539   0.21681  0.098428   
Weather     0.125       0.025955  0.047594  0.026424  0.047342  0.026092   
            0.25        0.029656  0.054529  0.029405  0.054105  0.029947   
            0.375       0.032986  0.060237  0.033181   0.06054  0.033569   
            0.5         0.037267  0.066907  0.037287    0.0665  0.037713   

                                     qlora            
                             MAE       MSE       MAE  
Dataset     Mask Ratio                                
ETTh1       0.125       0.164925  0.052166  0.153412  
            0.25        0.193981  0.072907   0.17984  
            0.375       0.220696   0.09961   0.20796  
            0.5          0.24721   0.12912  0.235553  
ETTh2       0.125       0.136537  0.041976  0.131979  
            0.25        0.151722   0.04923  0.145407  
            0.375       0.163616  0.057515   0.15964  
            0.5         0.175587   0.06657  0.171865  
ETTm1       0.125       0.113787  0.023808  0.103594  
            0.25         0.13057  0.032801  0.121337  
            0.375       0.148138  0.041271  0.135434  
            0.5          0.16497  0.052642  0.152041  
ETTm2       0.125       0.091366  0.020969  0.086978  
            0.25        0.100163   0.02464  0.096534  
            0.375       0.107905  0.028002  0.103877  
            0.5         0.116362  0.031611  0.111122  
Electricity 0.125        0.18707   0.07141  0.185886  
            0.25         0.19894  0.080468  0.197021  
            0.375       0.208935  0.088234  0.206479  
            0.5          0.21942  0.096641  0.216725  
Weather     0.125       0.047484  0.026301   0.04781  
            0.25        0.055123  0.030199  0.055308  
            0.375       0.061388  0.033979  0.061814  
            0.5         0.066793  0.037499  0.066783

In [27]:
peft_tuning_results_agg

lora             adalora                 ia3            \
                  MSE       MAE       MSE       MAE       MSE       MAE   
Dataset                                                                   
ETTh1        0.086692  0.193787  0.091508  0.197315  0.100542  0.206703   
ETTh2        0.053755    0.1519  0.054213  0.152823  0.056412  0.156865   
ETTm1        0.037961  0.128528   0.04137   0.13386  0.045222  0.139366   
ETTm2         0.02636  0.099961  0.026988  0.101401  0.028039  0.103949   
Electricity  0.084088  0.201328  0.084213  0.201671  0.085578  0.203591   
Weather      0.031466  0.057317  0.031574  0.057122   0.03183  0.057697   

                qlora            
                  MSE       MAE  
Dataset                          
ETTh1        0.088451  0.194192  
ETTh2        0.053823  0.152223  
ETTm1         0.03763  0.128102  
ETTm2        0.026305  0.099628  
Electricity  0.084188  0.201528  
Weather      0.031994  0.057929

In [28]:
peft_params[:1]

lora                                 adalora                   \
       all_params trainable_params percentage  all_params trainable_params   
ETTh1  60759552.0         906240.0       1.49  60814926.0         961608.0   

                         ia3                                   qlora  \
      percentage  all_params trainable_params percentage  all_params   
ETTh1       1.58  60658176.0         795648.0       1.31  60759552.0   

                                   
      trainable_params percentage  
ETTh1         906240.0       1.49